In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install torch
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from networks import *


In [ ]:
from model_eval import *
from model_prep import * 
from model_training import *

In [ ]:
import os
print(os.getcwd())


In [ ]:
#data1 = pd.read_csv("SML2010/NEW-DATA-1.T15.txt", sep=' ')
#data2 = pd.read_csv("SML2010/NEW-DATA-2.T15.txt", sep=' ')
df = pd.read_csv("data/weather_int.csv")

In [ ]:
df = prepare_datetime_index(data3, timestamp_col='date', drop_original=True)

In [ ]:
df.columns

In [ ]:
target_weather= 'T (degC)'
cols_weather= ['p (mbar)', 'Tdew (degC)', 'sh (g/kg)', 'wv (m/s)', 'max. wv (m/s)',
       'wd (deg)', 'T (degC)']

In [ ]:
df.info()

In [ ]:
#original config
train_size = 3200
val_size = 400
depth = 10
batch_size = 128
prediction_horizon = 1

In [ ]:
X_train1 = np.zeros((len(data1), depth, len(cols)))
y_train1 = np.zeros((len(data1), 1))

In [ ]:
for i, name in enumerate(cols):
    for j in range(depth):
        X_train1[:, j, i] = data1[name].shift(depth - j - 1).fillna(method="bfill")
y_train1 = data1[target].shift(-prediction_horizon).fillna(method='ffill')

In [ ]:
X_train1 = X_train1[depth:-prediction_horizon]
y_train1 = y_train1[depth:-prediction_horizon]

In [ ]:
X2 = np.zeros((len(data2), depth, len(cols)))
y2 = np.zeros((len(data2), 1))

In [ ]:
for i, name in enumerate(cols):
    for j in range(depth):
        X2[:, j, i] = data2[name].shift(depth - j - 1).fillna(method="bfill")
y2 = data2[target].shift(-prediction_horizon).fillna(method='ffill')

In [ ]:
X_train2 = X2[:train_size - len(data1)]
y_train2 = y2[:train_size - len(data1)]

X_val = X2[train_size - len(data1):train_size - len(data1) + val_size]
y_val = y2[train_size - len(data1):train_size - len(data1) + val_size]

X_test = X2[train_size - len(data1) + val_size:]
y_test = y2[train_size - len(data1) + val_size:]

In [ ]:
X_train2 = X_train2[depth:]
y_train2 = y_train2[depth:]

In [ ]:
X_train = np.concatenate([X_train1, X_train2], axis=0)
y_train = np.concatenate([y_train1, y_train2], axis=0)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_train_min, y_train_min = X_train.min(axis=0), y_train.min(axis=0)
X_train_max, y_train_max = X_train.max(axis=0), y_train.max(axis=0)

In [ ]:
X_train = (X_train - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_val = (X_val - X_train_min)/(X_train_max - X_train_min + 1e-9)
X_test = (X_test - X_train_min)/(X_train_max - X_train_min + 1e-9)

y_train = (y_train - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_val = (y_val - y_train_min)/(y_train_max - y_train_min + 1e-9)
y_test = (y_test - y_train_min)/(y_train_max - y_train_min + 1e-9)


In [ ]:
X_train_t = torch.Tensor(X_train)
X_val_t = torch.Tensor(X_val)
X_test_t = torch.Tensor(X_test)
y_train_t = torch.Tensor(y_train)
y_val_t = torch.Tensor(y_val.values)
y_test_t = torch.Tensor(y_test.values)

In [ ]:
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), shuffle=True, batch_size=batch_size)
val_loader = DataLoader(TensorDataset(X_val_t, y_val_t), shuffle=False, batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), shuffle=False, batch_size=batch_size)


In [ ]:
import numpy as np

# Parameters (similar to your old setup)
#first run (works well)
#depth = 10
#batch_size = 128
#prediction_horizon = 1
#2nd run
depth_weather = 10
batch_size_weather = 128
prediction_horizon_weather = 1

# Suppose you have these
#cols_weather = [...]  # your input features (list of column names)
#target_weather = 'your_target_column'  # your target column name

# Size splits
train_size_weather = int(0.7 * len(df))   # 70% train
val_size_weather = int(0.15 * len(df))    # 15% validation
test_size_weather = len(df) - train_size_weather - val_size_weather  # rest for testing

# Create input X and y
X_weather = np.zeros((len(df), depth_weather, len(cols_weather)))
y_weather = np.zeros((len(df), 1))

for i, name in enumerate(cols_weather):
    for j in range(depth_weather):
        X_weather[:, j, i] = df[name].shift(depth_weather - j - 1).fillna(method="bfill")

y_weather = df[target_weather].shift(-prediction_horizon_weather).fillna(method='ffill')

# Remove the first `depth_weather` samples and last `prediction_horizon_weather` samples
X_weather = X_weather[depth_weather:-prediction_horizon_weather]
y_weather = y_weather[depth_weather:-prediction_horizon_weather]

# Now split into train/val/test
X_train_weather = X_weather[:train_size_weather - depth_weather]
y_train_weather = y_weather[:train_size_weather - depth_weather]

X_val_weather = X_weather[train_size_weather - depth_weather:train_size_weather + val_size_weather - depth_weather]
y_val_weather = y_weather[train_size_weather - depth_weather:train_size_weather + val_size_weather - depth_weather]

X_test_weather = X_weather[train_size_weather + val_size_weather - depth_weather:]
y_test_weather = y_weather[train_size_weather + val_size_weather - depth_weather:]


In [ ]:
X_train_weather.shape, y_train_weather.shape

In [ ]:
X_train_weather_min, y_train_weather_min = X_train_weather.min(axis=0), y_train_weather.min(axis=0)
X_train_weather_max, y_train_weather_max = X_train_weather.max(axis=0), y_train_weather.max(axis=0)

In [ ]:
X_train_weather = (X_train_weather - X_train_weather_min)/(X_train_weather_max - X_train_weather_min + 1e-9)
X_val_weather = (X_val_weather - X_train_weather_min)/(X_train_weather_max - X_train_weather_min + 1e-9)
X_test_weather = (X_test_weather - X_train_weather_min)/(X_train_weather_max - X_train_weather_min + 1e-9)

y_train_weather = (y_train_weather - y_train_weather_min)/(y_train_weather_max - y_train_weather_min + 1e-9)
y_val_weather = (y_val_weather - y_train_weather_min)/(y_train_weather_max - y_train_weather_min + 1e-9)
y_test_weather = (y_test_weather - y_train_weather_min)/(y_train_weather_max - y_train_weather_min + 1e-9)


In [ ]:
X_train_t_weather = torch.Tensor(X_train_weather)
X_val_t_weather = torch.Tensor(X_val_weather)
X_test_t_weather = torch.Tensor(X_test_weather)
y_train_t_weather = torch.Tensor(y_train_weather)
y_val_t_weather = torch.Tensor(y_val_weather.values)
y_test_t_weather = torch.Tensor(y_test_weather.values)

In [ ]:
train_loader_weather = DataLoader(TensorDataset(X_train_t_weather, y_train_t_weather), shuffle=True, batch_size=batch_size_weather)
val_loader_weather = DataLoader(TensorDataset(X_val_t_weather, y_val_t_weather), shuffle=False, batch_size=batch_size_weather)
test_loader_weather = DataLoader(TensorDataset(X_test_t_weather, y_test_t_weather), shuffle=False, batch_size=batch_size_weather)


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_weather = IMVTensorLSTM(X_train_weather.shape[2], 1, 128).to(device)
opt_weather = torch.optim.Adam(model_weather.parameters(), lr=0.01)

In [ ]:
epoch_scheduler_weather = torch.optim.lr_scheduler.StepLR(opt_weather, 20, gamma=0.9)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

In [ ]:
train_model_single_step(model=model_weather,
    train_loader=train_loader_weather,
    val_loader=val_loader_weather,
    optimizer=opt_weather,
    scheduler=epoch_scheduler_weather,
    loss_fn=torch.nn.MSELoss(),
    num_epochs=30,
    patience=35,
    save_path="weather_lstm.pt",
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    y_min=y_train_weather_min,
    y_max=y_train_weather_max,
    X_train_len=len(X_train_t_weather),
    X_val_len=len(X_val_t_weather)
)

In [ ]:
model = IMVTensorLSTM(X_train.shape[2], 1, 128).cuda()
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epoch_scheduler = torch.optim.lr_scheduler.StepLR(opt, 20, gamma=0.9)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
import time

In [ ]:
epochs = 1000
loss = nn.MSELoss()
patience = 35
min_val_loss = 9999
counter = 0
for i in range(epochs):
    mse_train = 0
    iteration_start = time.monotonic()
    for batch_x, batch_y in train_loader :
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        opt.zero_grad()
        y_pred, alphas, betas = model(batch_x)
        y_pred = y_pred.squeeze(1)
        l = loss(y_pred, batch_y)
        l.backward()
        mse_train += l.item()*batch_x.shape[0]
        opt.step()
    epoch_scheduler.step()
    with torch.no_grad():
        mse_val = 0
        preds = []
        true = []
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()
            output, alphas, betas = model(batch_x)
            output = output.squeeze(1)
            preds.append(output.detach().cpu().numpy())
            true.append(batch_y.detach().cpu().numpy())
            mse_val += loss(output, batch_y).item()*batch_x.shape[0]
    preds = np.concatenate(preds)
    true = np.concatenate(true)
    
    if min_val_loss > mse_val**0.5:
        min_val_loss = mse_val**0.5
        print("Saving...")
        torch.save(model.state_dict(), "imv_lstm_sml2010.pt")
        counter = 0
    else: 
        counter += 1
    
    if counter == patience:
        break
    print("Iter: ", i, "train: ", (mse_train/len(X_train_t))**0.5, "val: ", (mse_val/len(X_val_t))**0.5)
    iteration_end = time.monotonic()
    print("Iter time: ", iteration_end - iteration_start)
    if(i % 10 == 0):
        preds = preds*(y_train_max - y_train_min) + y_train_min
        true = true*(y_train_max - y_train_min) + y_train_min
        mse = mean_squared_error(true, preds)
        mae = mean_absolute_error(true, preds)
        print("mse: ", mse, "mae: ", mae)
        plt.figure(figsize=(20, 10))
        plt.plot(preds)
        plt.plot(true)
        plt.show()

# Eval weather

In [ ]:
model_weather.load_state_dict(torch.load("weather_lstm.pt"))

In [ ]:
with torch.no_grad():
    mse_val_weather = 0
    preds_weather = []
    true_weather = []
    alphas_weather = []
    betas_weather = []
    for batch_x, batch_y in test_loader_weather:
        # Move batch to the correct device (GPU or CPU)
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        # Forward pass through the model
        output_weather, a, b = model_weather(batch_x)
        output_weather = output_weather.squeeze(1)
    
        # Store predictions and ground truth
        preds_weather.append(output_weather.detach().cpu().numpy())
        true_weather.append(batch_y.detach().cpu().numpy())
    
        # Store alpha and beta values if needed
        alphas_weather.append(a.detach().cpu().numpy())
        betas_weather.append(b.detach().cpu().numpy())
    
        # Compute loss (you can use the same loss function you had before)
        mse_val_weather += loss_weather(output_weather, batch_y).item() * batch_x.shape[0]
preds_weather = np.concatenate(preds_weather)
true_weather = np.concatenate(true_weather)
alphas_weather = np.concatenate(alphas_weather)
betas_weather = np.concatenate(betas_weather)

In [ ]:
preds_weather = preds_weather*(y_train_weather_max - y_train_weather_min) + y_train_weather_min
true_weather = true_weather*(y_train_weather_max - y_train_weather_min) + y_train_weather_min

In [ ]:
mse_weather = mean_squared_error(true_weather, preds_weather)
mae_weather = mean_absolute_error(true_weather, preds_weather)

In [ ]:
mse_weather, mae_weather

In [ ]:
mse_weather**0.5

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(preds_weather)
plt.plot(true_weather)
plt.show()

In [ ]:
alphas_weather.shape

In [ ]:
betas_weather.shape

In [ ]:
alphas_weather = alphas_weather.mean(axis=0)
betas_weather = betas_weather.mean(axis=0)

In [ ]:
alphas_weather = alphas_weather[..., 0]
betas_weather = betas_weather[..., 0]

In [ ]:
alphas_weather = alphas_weather.transpose(1, 0)

In [ ]:
cols_weather

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im_weather = ax.imshow(alphas_weather)
ax.set_xticks(np.arange(X_train_t_weather.shape[1]))
ax.set_yticks(np.arange(len(cols_weather)))
ax.set_xticklabels(["t-" + str(i) for i in np.arange(X_train_t_weather.shape[1] - 1, -1, -1)])
ax.set_yticklabels(cols_weather)
for i in range(len(cols_weather)):
    for j in range(X_train_t_weather.shape[1]):
        text = ax.text(j, i, round(alphas_weather[i, j], 3),
                       ha="center", va="center", color="w")
ax.set_title("Importance of features and timesteps")
#fig.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.title("Feature importance")
plt.bar(range(len(cols_weather)), betas_weather)
plt.xticks(range(len(cols_weather)), cols_weather, rotation=90)
plt.show()

In [ ]:
with torch.no_grad():
    mse_val = 0
    preds = []
    true = []
    alphas = []
    betas = []
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.cuda()
        batch_y = batch_y.cuda()
        output, a, b = model(batch_x)
        output = output.squeeze(1)
        preds.append(output.detach().cpu().numpy())
        true.append(batch_y.detach().cpu().numpy())
        alphas.append(a.detach().cpu().numpy())
        betas.append(b.detach().cpu().numpy())
        mse_val += loss(output, batch_y).item()*batch_x.shape[0]
preds = np.concatenate(preds)
true = np.concatenate(true)
alphas = np.concatenate(alphas)
betas = np.concatenate(betas)

In [ ]:
preds = preds*(y_train_max - y_train_min) + y_train_min
true = true*(y_train_max - y_train_min) + y_train_min

In [ ]:
mse = mean_squared_error(true, preds)
mae = mean_absolute_error(true, preds)

In [ ]:
mse, mae

In [ ]:
mse**0.5

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(preds)
plt.plot(true)
plt.show()

In [ ]:
alphas.shape

In [ ]:
betas.shape

In [ ]:
alphas = alphas.mean(axis=0)
betas = betas.mean(axis=0)

In [ ]:
alphas = alphas[..., 0]
betas = betas[..., 0]

In [ ]:
alphas = alphas.transpose(1, 0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(alphas)
ax.set_xticks(np.arange(X_train_t.shape[1]))
ax.set_yticks(np.arange(len(cols)))
ax.set_xticklabels(["t-"+str(i) for i in np.arange(X_train_t.shape[1], -1, -1)])
ax.set_yticklabels(cols)
for i in range(len(cols)):
    for j in range(X_train_t.shape[1]):
        text = ax.text(j, i, round(alphas[i, j], 3),
                       ha="center", va="center", color="w")
ax.set_title("Importance of features and timesteps")
#fig.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.title("Feature importance")
plt.bar(range(len(cols)), betas)
plt.xticks(range(len(cols)), cols, rotation=90)

# Multi Step

In [ ]:
cols_weather_multi=['p (mbar)', 'Tdew (degC)', 'sh (g/kg)', 'wv (m/s)', 'max. wv (m/s)',
       'wd (deg)', 'T (degC)']

In [ ]:
df.info()

In [ ]:
input_window = 576   # instead of 40
forecast_horizon = 144

X_train_multi, y_train_multi, \
X_val_multi, y_val_multi, \
X_test_multi, y_test_multi, \
input_scaler_multi, target_scaler_multi = prepare_multistep_data(
    df=df,
    input_columns=cols_weather_multi,
    target_column=target_weather,
    input_window=input_window,
    forecast_horizon=forecast_horizon,
    scale_data=True
)


In [ ]:
X_train_t_multi = torch.tensor(X_train_multi, dtype=torch.float32)
X_val_t_multi   = torch.tensor(X_val_multi, dtype=torch.float32)
X_test_t_multi  = torch.tensor(X_test_multi, dtype=torch.float32)

y_train_t_multi = torch.tensor(y_train_multi, dtype=torch.float32)
y_val_t_multi   = torch.tensor(y_val_multi, dtype=torch.float32)
y_test_t_multi  = torch.tensor(y_test_multi, dtype=torch.float32)


In [ ]:
train_dataset=TensorDataset(X_train_t_multi, y_train_t_multi)
val_dataset=TensorDataset(X_val_t_multi, y_val_t_multi)
test_dataset=TensorDataset(X_test_t_multi, y_test_t_multi)

In [ ]:
from itertools import product
param_grid = {
    'n_units': [128,256],
    'lr': [1e-3, 1e-2],
    'forecast_steps': [10],
    'batch_size': [128]
   # 'attn_dim' :[16, 32]
}

# Create all combinations
param_combinations = list(product(*param_grid.values()))
param_keys = list(param_grid.keys())

In [ ]:
def grid_search_model(params, input_dim, output_dim, train_dataset, val_dataset, test_dataset, device):
    param_dict = dict(zip(param_keys, params))
    
    n_units = param_dict['n_units']
    lr = param_dict['lr']
    forecast_steps = param_dict['forecast_steps']
    batch_size = param_dict['batch_size']
   # attn_dim = param_dict['attn_dim']

    # Correct DataLoaders
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

    model = IMVTensorMultiStepLSTM(
        input_dim=input_dim,
        output_dim=output_dim,
        n_units=n_units,
        forecast_steps=forecast_steps
        #attn_dim=attn_dim
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)
    loss_fn = nn.MSELoss()

    model = train_multistep_model(
    model,
    train_loader,
    val_loader,
    num_epochs=30,
    learning_rate=lr,
    device=device
)

    # Now evaluate val and test loss
    model.eval()
    val_loss, test_loss = 0.0, 0.0
    loss_fn = nn.MSELoss()
    
    with torch.no_grad():
        # Validation loss
        total_val_loss, total_val_samples = 0, 0
        for X_val, y_val in val_loader:
            X_val, y_val = X_val.to(device), y_val.to(device)
            preds, _, _ = model(X_val)
            loss = loss_fn(preds.squeeze(-1), y_val)
            total_val_loss += loss.item() * X_val.size(0)
            total_val_samples += X_val.size(0)
        val_loss = total_val_loss / total_val_samples
    
        # Test loss
        total_test_loss, total_test_samples = 0, 0
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            preds, _, _ = model(X_test)
            loss = loss_fn(preds.squeeze(-1), y_test)
            total_test_loss += loss.item() * X_test.size(0)
            total_test_samples += X_test.size(0)
        test_loss = total_test_loss / total_test_samples
    
    return val_loss, test_loss, param_dict
    



In [ ]:
best_val_loss = float('inf')
best_params = None
best_test_loss = None

for params in param_combinations:
    try:
        val_loss, test_loss, param_dict = grid_search_model(
            params,
            input_dim=len(cols_weather_multi),
            output_dim=1,
            train_dataset=TensorDataset(X_train_t_multi, y_train_t_multi),
            val_dataset=TensorDataset(X_val_t_multi, y_val_t_multi),
            test_dataset=TensorDataset(X_test_t_multi, y_test_t_multi),
            device=device
        )
        print(f"Params: {param_dict} | Val Loss: {val_loss:.4f} | Test Loss: {test_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_test_loss = test_loss
            best_params = param_dict

    except Exception as e:
        print(f"Failed with params {params}: {e}")


In [ ]:
print("✅ Best Parameters Found:")
print(best_params)
print(f"Validation Loss: {best_val_loss:.4f}")
print(f"Test Loss with Best Params: {best_test_loss:.4f}")
Params: {'n_units': 128, 'lr': 0.01, 'forecast_steps': 10, 'batch_size': 128} 

In [ ]:

# Assuming you already have the following model initialized:
forecast_horizon=10

model_IMV_Con = IMVTensorMultiStepLSTMConicity(input_dim=len(cols_weather_multi), output_dim=1, n_units=128, forecast_steps=forecast_horizon)
num_epochs = 20  # You can adjust this as needed
learning_rate = 0.01  # Adjust based on experimentation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available


In [ ]:
trained_model_con = train_concocity_model(
    model=model_IMV_Con,
    train_loader=train_loader_multi,
    val_loader=val_loader_multi,
    num_epochs=20,
    learning_rate=0.01,
    device=device,
    target_scaler=target_scaler_multi  # Optional
)


In [ ]:
def evaluate_conicity_model(model, data_loader, device, target_scaler=None, print_metrics=True):
    model.eval()
    predictions = []
    targets = []
    alphas_all = []
    betas_all = []
    conicity_scores = []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred, alphas, betas, conicity = model(X_batch)

            predictions.append(y_pred.cpu())
            targets.append(y_batch.cpu())
            alphas_all.append(alphas.cpu())
            betas_all.append(betas.cpu())
            conicity_scores.append(conicity.item())

    predictions = torch.cat(predictions).numpy()
    targets = torch.cat(targets).numpy()
    alphas_all = torch.cat(alphas_all).numpy()
    betas_all = torch.cat(betas_all).numpy()

    # Optional inverse scaling
    if target_scaler is not None:
        from copy import deepcopy

        def inverse_scale(y):
            if y.ndim == 3 and y.shape[-1] == 1:
                y = y.squeeze(-1)
            y_flat = y.reshape(-1, target_scaler.mean_.shape[0])
            return target_scaler.inverse_transform(y_flat).reshape(y.shape)

        predictions = inverse_scale(predictions)
        targets = inverse_scale(targets)

    # Compute metrics
    mae = mean_absolute_error(targets.flatten(), predictions.flatten())
    rmse = mean_squared_error(targets.flatten(), predictions.flatten(), squared=False)
    r2 = r2_score(targets.flatten(), predictions.flatten())
    avg_conicity = np.mean(conicity_scores)

    if print_metrics:
        print(f"MAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f} | Avg Conicity: {avg_conicity:.4f}")

    return predictions, targets, alphas_all, betas_all, {
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "Conicity": avg_conicity
    }


In [ ]:
preds, trues, alphas, betas, metrics = evaluate_conicity_model(
    trained_model_con, test_loader_multi, device, target_scaler=target_scaler_multi
)


In [ ]:
import matplotlib.pyplot as plt

def plot_forecast_vs_truth(preds, trues, index=0):
    plt.figure(figsize=(10, 4))
    plt.plot(trues[index], label='Ground Truth')
    plt.plot(preds[index], label='Prediction')
    plt.title(f"Forecast vs. Ground Truth (Sample {index})")
    plt.xlabel("Forecast Step")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
import seaborn as sns

def plot_alpha_heatmap(alphas, sample_index=0):
    alpha = alphas[sample_index]  # shape: [seq_len, input_dim, n_units]
    alpha_mean = alpha.mean(axis=-1)  # shape: [seq_len, input_dim]

    plt.figure(figsize=(8, 6))
    sns.heatmap(alpha_mean.T, cmap='viridis', cbar=True)
    plt.title(f"Alpha Attention Heatmap (Sample {sample_index})")
    plt.xlabel("Time Step")
    plt.ylabel("Input Feature")
    plt.show()


In [ ]:
def plot_beta_per_step(betas, sample_index=0):
    beta = betas[sample_index]  # shape: [forecast_steps, input_dim]
    plt.figure(figsize=(10, 4))
    for i in range(beta.shape[1]):
        plt.plot(beta[:, i], label=f"Feature {i}")
    plt.title(f"Beta Attention Across Forecast Steps (Sample {sample_index})")
    plt.xlabel("Forecast Step")
    plt.ylabel("Attention Weight")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
def plot_conicity_distribution(conicity_scores):
    plt.figure(figsize=(6, 4))
    plt.hist(conicity_scores, bins=20, edgecolor='black')
    plt.title("Distribution of Conicity Scores")
    plt.xlabel("Conicity")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()


In [ ]:
def plot_conicity_over_epochs(conicity_list):
    plt.figure(figsize=(8, 4))
    plt.plot(conicity_list)
    plt.title("Mean Conicity Over Epochs")
    plt.xlabel("Epoch")
    plt.ylabel("Conicity")
    plt.grid(True)
    plt.show()


In [ ]:
plot_forecast_vs_truth(preds, trues, index=0)


In [ ]:
plot_alpha_heatmap(alphas, sample_index=0)


In [ ]:
plot_beta_per_step(betas, sample_index=0)


In [ ]:
conicity_list = []
with torch.no_grad():
    for X_batch, y_batch in test_loader_multi:
        y_pred, alphas, betas, mean_conicity =model_IMV_Con(X_batch.to(device))
        conicity_list.append(mean_conicity.item())


In [ ]:
plot_conicity_distribution(conicity_list)


In [ ]:
# First, define your loss function builder (from previous steps)
def build_loss_function(use_attention_regularization=False, attn_lambda=0.1):
    mse_loss = nn.MSELoss()
    
    def loss_fn(pred, target, attn_weights=None):
        loss = mse_loss(pred, target)
        if use_attention_regularization and attn_weights is not None:
            var = attn_weights.var(dim=1).mean()
            loss = loss + attn_lambda * var
        return loss
    
    return loss_fn
forecast_horizon=10
# Instantiate your model (make sure it returns attention weights)
model_reg = IMVTensorMultiStepLSTM(input_dim=len(cols_weather_multi), output_dim=1, n_units=128, forecast_steps=forecast_horizon)

# Build the loss function with attention regularization enabled
loss_fn = build_loss_function(use_attention_regularization=True, attn_lambda=0.1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
# Now run training
trained_model = train_multistep_model_reg(
    model_reg,
    train_loader_multi,
    val_loader_multi,
    num_epochs=20,
    learning_rate=1e-3,
    device=device,
    loss_fn=loss_fn,
    target_scaler=target_scaler_multi
)


In [ ]:
def evaluate_reg_model(model, data_loader, device, target_scaler=None, print_metrics=True):
    model.eval()
    predictions = []
    targets = []
    alphas_all = []
    betas_all = []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred, alphas, betas, _ = model(X_batch)

            predictions.append(y_pred.cpu())
            targets.append(y_batch.cpu())
            alphas_all.append(alphas.cpu())
            betas_all.append(betas.cpu())

    predictions = torch.cat(predictions).numpy()
    targets = torch.cat(targets).numpy()
    alphas_all = torch.cat(alphas_all).numpy()
    betas_all = torch.cat(betas_all).numpy()

    # Optional inverse scaling
    if target_scaler is not None:
        from copy import deepcopy

        def inverse_scale(y):
            if y.ndim == 3 and y.shape[-1] == 1:
                y = y.squeeze(-1)
            y_flat = y.reshape(-1, target_scaler.mean_.shape[0])
            return target_scaler.inverse_transform(y_flat).reshape(y.shape)

        predictions = inverse_scale(predictions)
        targets = inverse_scale(targets)

    # Compute metrics
    mae = mean_absolute_error(targets.flatten(), predictions.flatten())
    rmse = mean_squared_error(targets.flatten(), predictions.flatten(), squared=False)
    r2 = r2_score(targets.flatten(), predictions.flatten())

    if print_metrics:
        print(f"MAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f}")

    return predictions, targets, alphas_all, betas_all, {"MAE": mae, "RMSE": rmse, "R2": r2}

In [ ]:
preds, trues, alphas, betas, metrics = evaluate_reg_model(
    trained_model, test_loader_multi, device, target_scaler=target_scaler_multi
)

# Regular IMVLSTM

In [ ]:
batch_size_weather = 128
#prediction_horizon_weather = 1
# Training, validation, and test DataLoaders for multistep forecasting
train_loader = DataLoader(
    TensorDataset(X_train_t_multi, y_train_t_multi),
    shuffle=True,
    batch_size=batch_size_weather
)

val_loader = DataLoader(
    TensorDataset(X_val_t_multi, y_val_t_multi),
    shuffle=False,
    batch_size=batch_size_weather
)

test_loader = DataLoader(
    TensorDataset(X_test_t_multi, y_test_t_multi),
    shuffle=False,
    batch_size=batch_size_weather
)

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_IMV_model(
    model,
    data_loader: DataLoader,
    device,
    target_scaler=None,
    save_attention: bool = False,
    save_dir: str = "./attention_test",
    print_metrics: bool = True
):
    model.eval()
    preds_list, trues_list = [], []
    alphas_list, betas_list = [], []

    with torch.no_grad():
        for Xb, yb in data_loader:
            Xb = Xb.to(device)
            yb = yb.to(device)

            y_pred, alphas, betas, _ = model(Xb)    # y_pred: [B, H, 1]
            preds_list.append(y_pred.cpu())
            trues_list.append(yb.cpu())
            alphas_list.append(alphas.cpu())
            betas_list.append(betas.cpu())

    preds = torch.cat(preds_list).numpy()    # (N, H, 1)
    trues = torch.cat(trues_list).numpy()    # (N, H)
    alphas = torch.cat(alphas_list).numpy()  # (N, seq_len, D, 1)
    betas  = torch.cat(betas_list).numpy()   # (N, H, D, 1)

    # 1) Squeeze out that last singleton dimension
    preds2 = preds.squeeze(-1)  # → (N, H)
    # trues are already (N, H) from your DataLoader

    # 2) Inverse‐scale if a scaler was provided
    if target_scaler is not None:
        # This assumes target_scaler was fit on shape (N_train, H)
        preds2 = target_scaler.inverse_transform(preds2)
        trues = target_scaler.inverse_transform(trues)

    # 3) Save the raw attention (before flattening) if requested
    if save_attention:
        os.makedirs(save_dir, exist_ok=True)
        np.save(os.path.join(save_dir, "alphas.npy"), alphas)
        np.save(os.path.join(save_dir, "betas.npy"),  betas)
        if print_metrics:
            print(f"✅ Saved attention maps to {save_dir}/{{alphas.npy,betas.npy}}")

    # 4) Compute metrics
    y_pred_flat = preds2.reshape(-1)
    y_true_flat = trues.reshape(-1)

    mae  = mean_absolute_error(y_true_flat, y_pred_flat)
    rmse = mean_squared_error(y_true_flat, y_pred_flat)
    r2   = r2_score(y_true_flat, y_pred_flat)

    if print_metrics:
        print(f"→ Eval: MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")

    return preds2, trues, alphas, betas, {"MAE": mae, "RMSE": rmse, "R2": r2}


def train_and_evaluate_IMV(
    model,
    train_loader: DataLoader,
    val_loader: DataLoader,
    test_loader: DataLoader,
    device,
    target_scaler,
    num_epochs: int = 100,
    lr: float = 3e-4,
    patience: int = 25,
    save_attention_dir: str = "./imv_attention"
):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    best_val_rmse = float("inf")
    patience_ctr = 0

    for epoch in range(1, num_epochs+1):
        model.train()
        train_losses = []
        all_preds, all_trues = [], []

        print(f"\n=== Epoch {epoch}/{num_epochs} ===")

        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            optimizer.zero_grad()

            y_pred, _, _, _ = model(Xb)       # [B, H, 1]
            y_pred = y_pred.squeeze(-1)       # [B, H]

            loss = criterion(y_pred, yb)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item() * Xb.size(0))
            all_preds.append(y_pred.detach().cpu().numpy())
            all_trues.append(yb.detach().cpu().numpy())

        # Aggregate training metrics
        avg_train_loss = sum(train_losses) / len(train_loader.dataset)
        preds_flat = np.concatenate(all_preds, axis=0).reshape(-1)
        trues_flat = np.concatenate(all_trues, axis=0).reshape(-1)

        train_mae  = mean_absolute_error(trues_flat, preds_flat)
        train_rmse = mean_squared_error(trues_flat, preds_flat)
        train_r2   = r2_score(trues_flat, preds_flat)

        print(f"Train → Loss: {avg_train_loss:.4f}, MAE: {train_mae:.4f}, "
              f"RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}")

        # ——— Validation ———
        _, _, _, _, val_metrics = evaluate_IMV_model(
            model, val_loader, device,
            target_scaler=target_scaler,
            save_attention=False,
            print_metrics=True
        )
        val_rmse = val_metrics["RMSE"]

        # Early stopping logic
        if val_rmse < best_val_rmse * 0.995:  # require 0.5% improvement
            best_val_rmse = val_rmse
            patience_ctr = 0
            torch.save(model.state_dict(), "imv_best.pth")
            print("🎉 New best model saved.")
        else:
            patience_ctr += 1
            print(f"No improvement → patience {patience_ctr}/{patience}")
            if patience_ctr >= patience:
                print("⏱ Early stopping.")
                break

    # Load best and do final eval on test set (saving its attention maps)
    model.load_state_dict(torch.load("imv_best.pth"))
    return evaluate_IMV_model(
        model,
        test_loader,
        device,
        target_scaler=target_scaler,
        save_attention=True,
        save_dir=save_attention_dir,
        print_metrics=True
    )


In [ ]:

# 1) Instantiate & move to device
model_IMV = IMVTensorMultiStepLSTM(
    input_dim    = len(cols_weather_multi),  # number of input vars
    output_dim   = 1,                        # forecasting one target
    n_units      = 140,                      # hidden dimension
    forecast_steps = 144                      # 10-step horizon
).to(device)

# 1) Train + save best IMV‐LSTM on val‐split (and save attention for val)
preds_val, trues_val, alphas_val, betas_val, metrics_val = train_and_evaluate_IMV(
    model_IMV,
    train_loader,
    val_loader,
    test_loader,
    device,
    target_scaler_multi,
    num_epochs=100,
    lr=3e-4,
    patience=25,
    save_attention_dir="./imv_attention_val"   # saves alphas.npy & betas.npy for val
)

# 2) Now evaluate on your test set
preds_test, trues_test, alphas_test, betas_test, metrics_test = evaluate_IMV_model(
    model_IMV,
    test_loader,
    device,
    target_scaler=target_scaler_multi,
    save_attention=True,          # save the test‐set attention maps too
    save_dir="./imv_attention_test",
    print_metrics=True
)

# 3) metrics_test now holds your test MAE/RMSE/R²
print("Test Metrics:", metrics_test)


In [2]:
%run scripts/PatchTST/weather_int_IMV_LSTM.py \
  --root_path C:/Users/miche/Documents/PatchTST/PatchTST_supervised/dataset/ \
  --data_path weather_int.csv \
  --input_window 576 \
  --forecast_horizon 144 \
  --batch_size 128 \
  --epochs 100 \
  --lr 3e-4 \
  --patience 25 \
  --save_dir ./logs/IMV_weather



=== Epoch 1/1 ===
Train → Loss: 0.4910, MAE: 0.5554, RMSE: 0.4910, R²: 0.5090
→ Eval: MAE=4.5679, RMSE=30.7695, R²=-0.1433
🎉 New best model saved.
✅ Saved attention maps to ./logs/IMV_weather\attention_val/{alphas.npy,betas.npy}
→ Eval: MAE=3.7295, RMSE=18.9554, R²=-0.3037
✅ Saved attention maps to ./logs/IMV_weather\attention_test/{alphas.npy,betas.npy}
→ Eval: MAE=3.7295, RMSE=18.9554, R²=-0.3037
Test metrics: {'MAE': 3.729501724243164, 'RMSE': 18.955427169799805, 'R2': -0.30368101596832275}


# Direct multi-step forecasting

In [ ]:
import torch.nn.functional as F

def train_direct_imv_model(
    model,
    train_loader,
    val_loader,
    num_epochs=30,
    lr=1e-3,
    weight_decay=1e-5,
    device="cuda" if torch.cuda.is_available() else "cpu"
):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()

    for epoch in range(1, num_epochs + 1):
        model.train()
        train_losses, train_mae, train_rmse = [], [], []

        for xb, yb in tqdm(train_loader, desc=f"[Epoch {epoch:03d}] Training"):
            xb, yb = xb.to(device), yb.to(device)

            optimizer.zero_grad()
            y_pred, _, _, _ = model(xb)
            loss = criterion(y_pred, yb)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            train_mae.append(torch.mean(torch.abs(y_pred - yb)).item())
            train_rmse.append(torch.sqrt(F.mse_loss(y_pred, yb)).item())

        # Validation
        model.eval()
        val_losses, val_mae, val_rmse = [], [], []

        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                y_pred, _, _, _ = model(xb)
                loss = criterion(y_pred, yb)

                val_losses.append(loss.item())
                val_mae.append(torch.mean(torch.abs(y_pred - yb)).item())
                val_rmse.append(torch.sqrt(F.mse_loss(y_pred, yb)).item())

        print(
            f"Epoch {epoch:03d} | "
            f"Train Loss: {np.mean(train_losses):.4f} | Val Loss: {np.mean(val_losses):.4f} | "
            f"Train MAE: {np.mean(train_mae):.4f} | Val MAE: {np.mean(val_mae):.4f} | "
            f"Train RMSE: {np.mean(train_rmse):.4f} | Val RMSE: {np.mean(val_rmse):.4f}"
        )

    return model


In [ ]:
X_train_multi_dir, y_train_multi_dir, \
X_val_multi_dir, y_val_multi_dir, \
X_test_multi_dir, y_test_multi_dir, \
input_scaler_multi_dir, target_scaler_multi_dir = prepare_multistep_data(
    df=df,
    input_columns=cols_weather_multi,
    target_column=target_weather,
    input_window=10,
    forecast_horizon=3,
    scale_data=True
)


In [ ]:
X_train_t_multi_dir = torch.tensor(X_train_multi_dir, dtype=torch.float32)
X_val_t_multi_dir   = torch.tensor(X_val_multi_dir, dtype=torch.float32)
X_test_t_multi_dir  = torch.tensor(X_test_multi_dir, dtype=torch.float32)

y_train_t_multi_dir = torch.tensor(y_train_multi_dir, dtype=torch.float32)
y_val_t_multi_dir   = torch.tensor(y_val_multi_dir, dtype=torch.float32)
y_test_t_multi_dir  = torch.tensor(y_test_multi_dir, dtype=torch.float32)


In [ ]:
#2nd run
#depth_weather = 10
batch_size_weather = 128
#prediction_horizon_weather = 1
# Training, validation, and test DataLoaders for multistep forecasting
train_loader_multi_dir = DataLoader(
    TensorDataset(X_train_t_multi_dir, y_train_t_multi_dir),
    shuffle=True,
    batch_size=batch_size_weather
)

val_loader_multi_dir = DataLoader(
    TensorDataset(X_val_t_multi_dir, y_val_t_multi_dir),
    shuffle=False,
    batch_size=batch_size_weather
)

test_loader_multi_dir = DataLoader(
    TensorDataset(X_test_t_multi_dir, y_test_t_multi_dir),
    shuffle=False,
    batch_size=batch_size_weather
)

In [ ]:
direct_model = DirectIMVTensorLSTM(
    input_dim=X_train_t_multi_dir.shape[2],  # number of input features
    output_dim=3,                        # forecast_horizon (i.e., 3 steps ahead)
    n_units=64                           # hidden units (tunable)
)


In [ ]:
trained_model = train_direct_imv_model(
    model=direct_model,
    train_loader=train_loader_multi_dir,
    val_loader=val_loader_multi_dir,
    num_epochs=20,
    lr=1e-3,
    weight_decay=1e-5
)


# visualisations

In [ ]:
import importlib
import model_prep
importlib.reload(model_prep)
# Assuming predictions and targets are scaled arrays, and target_scaler_multi is the fitted scaler
print("Before inverse scaling:")
print(f"Predictions shape: {predictions.shape}")
print(f"Targets shape: {targets.shape}")

preds_unscaled = inverse_scale_predictions(predictions, target_scaler_multi)
targets_unscaled = inverse_scale_predictions(targets, target_scaler_multi)

# Check the unscaled results' shape
print("After inverse scaling:")
print(f"Predictions unscaled shape: {preds_unscaled.shape}")
print(f"Targets unscaled shape: {targets_unscaled.shape}")

In [ ]:

# Now you can use the plotting functions to visualize:
plot_predictions(targets_unscaled, preds_unscaled, num_samples=10)

In [ ]:
plot_predictions(targets_unscaled, preds_unscaled, num_samples=5)


In [ ]:
plot_attention_weights(alphas, betas, sample_idx=0, input_var_names=cols_weather_multi)


In [ ]:
plot_beta_attention_over_forecast_steps(betas, sample_idx=0)


In [ ]:
plot_forecast_vs_actual_per_step(predictions, targets, sample_idx=0)


In [ ]:
plot_forecast_vs_actual_per_step(preds_unscaled, targets_unscaled, sample_idx=0)

In [ ]:
plot_average_forecast_vs_actual_per_step(preds_unscaled, targets_unscaled)


In [ ]:
plot_forecast_steps(preds_unscaled, targets_unscaled, forecast_horizon=10)


In [ ]:
plot_forecast_steps_with_error_bands(preds_unscaled, targets_unscaled, forecast_horizon=10, subset_size=10)


In [ ]:
# Example 1: Randomly select 10 samples
plot_forecast_steps_with_error_bands(preds_unscaled, targets_unscaled, forecast_horizon=forecast_horizon, subset_size=10, random_subset=True)

# Example 2: Manually select specific samples
#plot_forecast_steps_with_error_bands(predictions, targets, forecast_horizon=5, subset_size=10, selected_indices=[0, 5, 10])

# Example 3: Select samples based on a condition (e.g., t+1 > 0.5)
#plot_forecast_steps_with_error_bands(predictions, targets, forecast_horizon=5, subset_size=10, threshold=0.5)


In [ ]:
# Example 3: Select samples based on a condition (e.g., t+1 > 0.5)
plot_forecast_steps_with_error_bands_deviation(preds_unscaled, targets_unscaled, forecast_horizon=forecast_horizon, subset_size=10, deviation_threshold=0.5)


In [ ]:
# Example 1: Plot only instances where the mean deviation across forecast steps exceeds 0.5
#plot_forecast_steps_with_error_bands(predictions, targets, forecast_horizon=5, subset_size=10, deviation_threshold=0.5)

# Example 2: Plot only instances where the mean deviation across forecast steps exceeds 1.0
plot_forecast_steps_with_error_bands(preds_unscaled, targets_unscaled, forecast_horizon=10, subset_size=10)


In [ ]:
X_train_dele, y_train_dele, \
X_val_dele, y_val_dele, \
X_test_dele, y_test_dele, \
input_scaler_dele, target_scaler_dele = prepare_multistep_data(
    df=df,
    input_columns=cols_weather_multi,
    target_column=target_weather,
    input_window=14,
    forecast_horizon=10,
    scale_data=True
)


In [ ]:
X_train_t_dele = torch.tensor(X_train_dele, dtype=torch.float32)
X_val_t_dele   = torch.tensor(X_val_dele, dtype=torch.float32)
X_test_t_dele  = torch.tensor(X_test_dele, dtype=torch.float32)

y_train_t_dele = torch.tensor(y_train_dele, dtype=torch.float32)
y_val_t_dele   = torch.tensor(y_val_dele, dtype=torch.float32)
y_test_t_dele  = torch.tensor(y_test_dele, dtype=torch.float32)


In [ ]:
#2nd run
#depth_weather = 10
batch_size_dele = 128
#prediction_horizon_weather = 1
# Training, validation, and test DataLoaders for multistep forecasting
train_loader_dele = DataLoader(
    TensorDataset(X_train_t_dele, y_train_t_dele),
    shuffle=True,
    batch_size=batch_size_dele
)

val_loader_dele = DataLoader(
    TensorDataset(X_val_t_dele, y_val_t_dele),
    shuffle=False,
    batch_size=batch_size_dele
)

test_loader_dele = DataLoader(
    TensorDataset(X_test_t_dele, y_test_t_dele),
    shuffle=False,
    batch_size=batch_size_dele
)

In [ ]:
def evaluate_model_dele(model, test_loader, forecast_steps, target_scaler, device):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)

            pred, _, _ = model(x, device)
            pred = pred[:, -forecast_steps:, 0]  # shape: [batch_size, forecast_steps]

            all_preds.append(pred.cpu())
            all_targets.append(y.cpu())

    # Concatenate all batches
    all_preds = torch.cat(all_preds, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    # Inverse transform the scaled data
    all_preds_unscaled = inverse_scale_predictions(all_preds, target_scaler)
    all_targets_unscaled = inverse_scale_predictions(all_targets, target_scaler)

    # Compute metrics
    mse = mean_squared_error(all_targets_unscaled, all_preds_unscaled)
    mae = mean_absolute_error(all_targets_unscaled, all_preds_unscaled)
    rmse = np.sqrt(mse)

    print(f"Evaluation Metrics on Unscaled Test Data:")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")

    return all_preds_unscaled, all_targets_unscaled

In [ ]:
import torch

# Example params (you need to adjust based on your input data)
Delelstm_params = {
    'input_dim': X_train_t_dele.shape[2],     # e.g. number of variables in input data
    'n_units': 64,                 # hidden size for tensor LSTM
    'time_depth':X_train_t_dele.shape[1],  # how many time steps input has
    'output_dim': 1,               # typically 1 for forecasting a single variable
    'N_units': 64                  # hidden size for vanilla LSTM
}

short = 2  # or another appropriate threshold used in your logic

# Instantiate the model
model_delelstm = Delelstm(Delelstm_params, short)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_delelstm.to(device)

optimizer_delelstm = torch.optim.Adam(model_delelstm.parameters(), lr=0.001)
criterion_delelstm = torch.nn.MSELoss()  # or MAE, depending on your metric


In [ ]:
 X_train_t_dele.shape[1]

In [ ]:
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

num_epochs = 30
forecast_steps = 10

for epoch in range(num_epochs):
    model_delelstm.train()
    total_loss = 0
    all_preds = []
    all_targets = []

    loop = tqdm(train_loader_dele, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)
    
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        optimizer_delelstm.zero_grad()

        pred, unorm, _ = model_delelstm(x, device)
        pred = pred[:, -forecast_steps:, 0]  # [batch_size, forecast_steps]

        loss = criterion_delelstm(pred, y)
        loss.backward()
        optimizer_delelstm.step()

        total_loss += loss.item()
        
        # Collect predictions and targets for metrics
        all_preds.append(pred.detach().cpu().numpy())
        all_targets.append(y.detach().cpu().numpy())

        loop.set_postfix(loss=loss.item())

    # Compute metrics
    all_preds = np.concatenate(all_preds, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    mae = mean_absolute_error(all_targets, all_preds)
    rmse = mean_squared_error(all_targets, all_preds, squared=False)

    print(f"Epoch {epoch+1} | Loss: {total_loss:.4f} | MAE: {mae:.4f} | RMSE: {rmse:.4f}")


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

forecast_steps = 10
model_delelstm.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for x_test, y_test in test_loader_dele:
        x_test, y_test = x_test.to(device), y_test.to(device)

        pred, _, _ = model_delelstm(x_test, device)
        pred = pred[:, -forecast_steps:, 0]  # same slicing as training

        all_preds.append(pred.cpu().numpy())
        all_targets.append(y_test.cpu().numpy())

# Convert to numpy arrays
all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)

# Compute metrics
mse = mean_squared_error(all_targets, all_preds)
mae = mean_absolute_error(all_targets, all_preds)
rmse = np.sqrt(mse)

print(f"\n📊 Test Metrics:")
print(f"  - MAE:  {mae:.4f}")
print(f"  - RMSE: {rmse:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Plot the first sample in the test set
plt.plot(all_targets[0], label='True')
plt.plot(all_preds[0], label='Predicted')
plt.title("Forecast vs Actual (first test sample)")
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.legend()
plt.show()


In [ ]:
all_preds_unscaled, all_targets_unscaled = evaluate_model_dele(
    model_delelstm,
    test_loader_dele,
    forecast_steps=10,
    target_scaler=target_scaler_dele,
    device=device
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(all_targets_unscaled[0], label="True")
plt.plot(all_preds_unscaled[0], label="Predicted")
plt.title("Multistep Forecast (first test sample)")
plt.legend()
plt.show()


# Evaluation 

In [ ]:
from scipy.stats import spearmanr
import copy

def randomization_sanity_check(model, data_loader, device=device):
    model_random = copy.deepcopy(model).to(device)

    # Randomize model weights
    for param in model_random.parameters():
        if param.requires_grad:
            torch.nn.init.normal_(param, mean=0, std=0.1)

    model.eval()
    model_random.eval()

    similarities_alpha = []
    similarities_beta = []

    with torch.no_grad():
        for x_batch, _ in data_loader:
            x_batch = x_batch.to(device)

            _, alpha_orig, beta_orig = model(x_batch)
            _, alpha_rand, beta_rand = model_random(x_batch)

            # Flatten
            alpha_orig = alpha_orig.reshape(-1).to(device).numpy()
            alpha_rand = alpha_rand.reshape(-1).to(device).numpy()
            beta_orig = beta_orig.reshape(-1).to(device).numpy()
            beta_rand = beta_rand.reshape(-1).to(device).numpy()

            # Compute Spearman similarity
            s_alpha, _ = spearmanr(alpha_orig, alpha_rand)
            s_beta, _ = spearmanr(beta_orig, beta_rand)

            similarities_alpha.append(s_alpha)
            similarities_beta.append(s_beta)

    print(f"Avg Spearman alpha similarity: {sum(similarities_alpha)/len(similarities_alpha):.4f}")
    print(f"Avg Spearman beta similarity:  {sum(similarities_beta)/len(similarities_beta):.4f}")


In [ ]:
def deletion_check(model, data_loader, k_ratio=0.1, use='alpha', device=device):
    model.eval()
    criterion = nn.MSELoss()
    errors_full, errors_deleted = [], []

    with torch.no_grad():
        for x_batch, y_batch in data_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            forecasts, alphas, betas = model(x_batch)
            loss_full = criterion(forecasts.squeeze(-1), y_batch)
            errors_full.append(loss_full.item())

            # Determine importance
            if use == 'alpha':
                importance = alphas.mean(dim=1)  # average across time: [batch, vars, units=1]
            elif use == 'beta':
                importance = betas.mean(dim=(1, 2))  # average over steps and units
            else:
                raise ValueError("use must be 'alpha' or 'beta'")

            importance_scores = importance.detach().to(device).numpy()
            x_batch_mod = x_batch.clone()

            for i in range(x_batch.size(0)):
                imp = importance_scores[i]
                topk = int(len(imp) * k_ratio)
                topk_idx = imp.argsort()[-topk:]
                x_batch_mod[i, :, topk_idx] = 0.0  # zero out top-k variables

            forecasts_mod, _, _ = model(x_batch_mod)
            loss_deleted = criterion(forecasts_mod.squeeze(-1), y_batch)
            errors_deleted.append(loss_deleted.item())

    print(f"Original Avg Loss: {sum(errors_full)/len(errors_full):.4f}")
    print(f"After Deletion Avg Loss: {sum(errors_deleted)/len(errors_deleted):.4f}")


In [ ]:
import torch
import numpy as np

def random_deletion_sanity_check(model, data_loader, top_k=3, device='cuda'):
    model.eval()
    model.to(device)
    criterion = nn.MSELoss()
    total_loss = 0.0

    with torch.no_grad():
        for x_batch, y_batch in data_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Randomly select top_k variable indices per batch
            batch_size, seq_len, input_dim = x_batch.shape
            mask = torch.ones_like(x_batch)

            for i in range(batch_size):
                random_vars = torch.randperm(input_dim)[:top_k]
                mask[i, :, random_vars] = 0  # zero out random variables

            x_deleted = x_batch * mask  # apply mask
            forecasts, _, _ = model(x_deleted)
            loss = criterion(forecasts.squeeze(-1), y_batch)
            total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"🔀 Random Deletion Avg Loss: {avg_loss:.4f}")
    return avg_loss


In [ ]:
model_IMV

In [ ]:
# After training
#randomization_sanity_check(model_inter, test_loader_multi)

randomization_sanity_check(model_IMV, test_loader_multi)

Alpha attention (temporal variable importance) has very low similarity to a random model → this suggests your model has learned meaningful temporal patterns.

Beta attention (variable-level forecasting importance) has some structure, but moderate similarity suggests it may still be influenced by spurious patterns or has weaker interpretability than alpha.

In [ ]:
#deletion_check(model_inter, test_loader_multi, k_ratio=0.2, use='alpha')  # or use='beta'
deletion_check(model_IMV, test_loader_multi, k_ratio=0.2, use='alpha')  # or use='beta'

In [ ]:
deletion_check(model_IMV, test_loader_multi, k_ratio=0.2, use='beta')  # or use='beta'

In [ ]:
#random_deletion_sanity_check(model_inter, test_loader_multi, top_k=3, device=device)
random_deletion_sanity_check(model_IMV, test_loader_multi, top_k=3, device=device)

| Scenario            | Avg Loss | Interpretation                                                       |
| ------------------- | -------- | -------------------------------------------------------------------- |
| **Original**        | `0.0107` | Normal test loss using the model with learned attention weights.     |
| **After Deletion**  | `0.0129` | Loss after **removing** (zeroing or masking) top-attended variables. |
| **Random Deletion** | `0.0925` | Loss after **randomly** removing variables (not based on attention). |
1. Deletion Check Validity
Deleting top-attended variables increased the loss slightly (0.0107 → 0.0129).

Random deletions increased the loss significantly (0.0107 → 0.0925).

This suggests:

The model indeed relies more heavily on variables it gives high attention to.

But deleting them doesn't cause catastrophic failure → possibly due to redundancy in your input features (other variables can partially compensate).

2. Trustworthiness of Attention
The fact that attention-guided deletion hurts performance less than random deletion shows that attention is pointing to genuinely important variables.

If deleting top-attended variables had no effect, the attention would be untrustworthy.

If deleting them caused massive degradation, it might mean your model is overly reliant on just a few variables.

# Visualisation of Evaluation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_alpha_heatmap(alphas, var_names=None, title="Alpha Attention Heatmap"):
    # alphas shape: [batch, time, input_dim, 1] → squeeze last dim
    alpha_vals = alphas.squeeze(-1).mean(dim=0).detach().to(device).numpy()  # [time, input_dim]
    
    plt.figure(figsize=(10, 6))
    sns.heatmap(alpha_vals.T, cmap="YlGnBu", xticklabels=True, yticklabels=var_names if var_names else True)
    plt.xlabel("Time Step")
    plt.ylabel("Variable")
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_beta_heatmap(betas, var_names=None, title="Beta Attention Heatmap"):
    # betas shape: [batch, forecast_steps, input_dim, 1] → squeeze and average
    beta_vals = betas.squeeze(-1).mean(dim=0).detach().to(device).numpy()  # [forecast_steps, input_dim]

    plt.figure(figsize=(10, 6))
    sns.heatmap(beta_vals.T, cmap="OrRd", xticklabels=True, yticklabels=var_names if var_names else True)
    plt.xlabel("Forecast Step")
    plt.ylabel("Variable")
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [ ]:
# Forward pass on a test batch
x_batch, _ = next(iter(test_loader_multi))
x_batch = x_batch.to(device)
_, alphas, betas = model_IMV(x_batch)

plot_alpha_heatmap(alphas, var_names=cols_weather_multi)
plot_beta_heatmap(betas, var_names=cols_weather_multi)


In [ ]:
cols_weather_multi

In [ ]:
import torch
import numpy as np
from scipy.stats import spearmanr
from torch.nn.functional import mse_loss

def evaluate_stability_fidelity(model, data_loader, device='cpu', epsilon=1e-2):
    model.eval()
    stability_scores = []
    fidelity_scores = []

    with torch.no_grad():
        for x_batch, _ in data_loader:
            x_batch = x_batch.to(device)

            # Original predictions and explanations
            preds_orig, alphas_orig, _ = model(x_batch)

            # Perturb input slightly
            noise = torch.randn_like(x_batch) * epsilon
            x_perturbed = x_batch + noise

            # Perturbed predictions and explanations
            preds_perturbed, alphas_perturbed, _ = model(x_perturbed)

            # ========== Stability: Spearman similarity between alphas ==========
            for alpha_o, alpha_p in zip(alphas_orig, alphas_perturbed):
                s, _ = spearmanr(alpha_o.cpu().numpy().flatten(), alpha_p.cpu().numpy().flatten())
                if not np.isnan(s):  # Filter out NaNs from constant vectors
                    stability_scores.append(s)

            # ========== Fidelity: MSE between predictions ==========
            fidelity = mse_loss(preds_orig, preds_perturbed).item()
            fidelity_scores.append(fidelity)

    print(f"Avg Stability (Spearman): {np.mean(stability_scores):.4f}")
    print(f"Avg Fidelity (MSE):       {np.mean(fidelity_scores):.6f}")


In [ ]:
# Run on your test/val loader
evaluate_stability_fidelity(model_IMV, test_loader_multi, device=device,epsilon=1e-1)


In [ ]:
import torch
import numpy as np
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_similarity_at_epsilon(model, data_loader, epsilon, device='cpu'):
    model.eval()
    spearman_scores = []
    cosine_scores = []

    with torch.no_grad():
        for x_batch, _ in data_loader:
            x_batch = x_batch.to(device)

            # Original explanations (alphas)
            _, alphas_orig, _ = model(x_batch)

            # Perturb inputs
            noise = torch.randn_like(x_batch) * epsilon
            x_perturbed = x_batch + noise

            # Perturbed explanations
            _, alphas_perturbed, _ = model(x_perturbed)

            # Compare each example in batch
            for alpha_o, alpha_p in zip(alphas_orig, alphas_perturbed):
                alpha_o_np = alpha_o.cpu().numpy().flatten()
                alpha_p_np = alpha_p.cpu().numpy().flatten()

                # Spearman
                s, _ = spearmanr(alpha_o_np, alpha_p_np)
                if not np.isnan(s):
                    spearman_scores.append(s)

                # Cosine similarity requires 2D arrays: reshape (1, -1)
                cos_sim = cosine_similarity(alpha_o_np.reshape(1, -1), alpha_p_np.reshape(1, -1))[0][0]
                cosine_scores.append(cos_sim)

    return np.mean(spearman_scores), np.mean(cosine_scores)


In [ ]:
import matplotlib.pyplot as plt

epsilons = np.linspace(0, 0.2, 21)  # from 0 to 0.2 in 0.01 steps
spearman_means = []
cosine_means = []

for e in epsilons:
    sp, cs = evaluate_similarity_at_epsilon(model_IMV, test_loader_multi, epsilon=e, device=device)
    spearman_means.append(sp)
    cosine_means.append(cs)
    print(f"Epsilon={e:.3f}: Spearman={sp:.4f}, Cosine={cs:.4f}")


In [ ]:
def plot_interaction_heatmap(interactions, var_names=None, title="Variable Interaction Heatmap"):
    # interactions shape: [batch, time, input_dim, input_dim]
    # Average over batch and time for a summary heatmap
    interaction_avg = interactions.mean(dim=0).mean(dim=0).detach().to(device).numpy()  # [input_dim, input_dim]

    plt.figure(figsize=(10, 8))
    sns.heatmap(
        interaction_avg,
        cmap="coolwarm",
        xticklabels=var_names if var_names else True,
        yticklabels=var_names if var_names else True,
        square=True,
        annot=True,  # Optional: show numbers in cells
        fmt=".2f"
    )
    plt.title(title)
    plt.xlabel("Variable (Key)")
    plt.ylabel("Variable (Query)")
    plt.show()


In [ ]:
forecasts, alphas, betas, interaction_weights = model_inter(x_batch)


In [ ]:
plot_interaction_heatmap(interaction_weights, var_names=cols_weather_multi)


What does the interaction heatmap show?
The heatmap visualizes learned interaction weights between variables at each time step.

Each cell (i, j) represents the strength of interaction from variable j to variable i as learned by your attention mechanism.

Values are usually normalized attention scores or learned weights indicating how much one variable influences another in the model’s internal state.

Why are diagonal values not 1?
Not a simple correlation or identity matrix:
The interaction weights are learned parameters, not raw correlation coefficients or similarity scores. They are attention scores that reflect relative importance or influence rather than a strict self-correlation.

Values are normalized:
Attention scores (like your alphas or interaction matrix) are often normalized with softmax or a similar function. This means the sum across a row or column may be 1, but individual values won’t necessarily be 1.

Diagonal isn’t forced to be 1:
Unlike correlation matrices where the diagonal is 1 by definition (variable perfectly correlated with itself), attention weights don’t have to be maximal on the diagonal. The model can learn that some variables might be more influenced by other variables than by themselves at certain time steps.

Scale depends on the model’s learned patterns:
If the diagonal is lower than 1, it might mean the model found other variables more informative or that self-interaction is not as dominant.

How to interpret values like 0.02 to 0.35?
Values closer to 0 mean weak or no interaction between those variables.

Higher values (like 0.3-0.35) indicate stronger learned influence.

Check if rows sum to 1 (or close), which is typical with attention weights — the heatmap shows relative importance.

Additional tips:
Compare interactions over time: If your interaction weights have a time dimension, look at how these weights evolve.

Interpret in context: High interaction weight from variable A to B means the model is using variable A’s info when updating variable B’s hidden state or prediction at that step.

Plot diagonal separately if needed: To see self-influence more clearly.

In [ ]:
# interaction_weights shape: [batch, time, input_dim, input_dim]
avg_interactions_time = interaction_weights.mean(dim=0)  # shape: [time, input_dim, input_dim]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

time_steps_to_plot = [0, 5, 9]  # example timesteps

for t in time_steps_to_plot:
    plt.figure(figsize=(8, 6))
    sns.heatmap(avg_interactions_time[t].cpu().detach().numpy(), 
                xticklabels=cols_weather_multi, yticklabels=cols_weather_multi,
                cmap="viridis", vmin=0, vmax=avg_interactions_time.max().item())
    plt.title(f"Variable Interactions at Time Step {t}")
    plt.xlabel("From Variable")
    plt.ylabel("To Variable")
    plt.show()


In [ ]:
# For example, interaction from variable 2 to variable 5 over time
var_from, var_to = 2, 5
interaction_over_time = avg_interactions_time[:, var_to, var_from].cpu().detach().numpy()

plt.plot(interaction_over_time)
plt.title(f"Interaction Weight from Var {var_from} to Var {var_to} Over Time")
plt.xlabel("Time Step")
plt.ylabel("Interaction Strength")
plt.show()


In [ ]:
self_interactions = avg_interactions_time[:, torch.arange(avg_interactions_time.size(1)), torch.arange(avg_interactions_time.size(1))]
# or simpler:
self_interactions = avg_interactions_time.diagonal(dim1=1, dim2=2)  # shape: [time, input_dim]

for i, var_name in enumerate(cols_weather_multi):
    plt.plot(self_interactions[:, i].cpu().detach().numpy(), label=var_name)

plt.title("Self-Interaction Strength Over Time")
plt.xlabel("Time Step")
plt.ylabel("Self-Interaction Strength")
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_interactions_from_variable(interactions_time, from_var_idx, var_names, figsize=(15, 10)):
    """
    Plot interaction weights over time from one variable to all others.

    Args:
        interactions_time: Tensor of shape [time, input_dim, input_dim] - averaged over batch
        from_var_idx: int - index of the variable from which interactions originate
        var_names: list of str - variable names matching input_dim
        figsize: tuple - figure size
    
    """
    time_steps = interactions_time.size(0)
    input_dim = interactions_time.size(1)

    n_cols = 4
    n_rows = int(np.ceil(input_dim / n_cols))

    plt.figure(figsize=figsize)
    
    for to_var_idx in range(input_dim):
        plt.subplot(n_rows, n_cols, to_var_idx + 1)
        y_vals = interactions_time[:, to_var_idx, from_var_idx].cpu().detach().numpy()
        plt.plot(range(time_steps), y_vals)
        plt.title(f"{var_names[from_var_idx]} → {var_names[to_var_idx]}")
        plt.xlabel("Time")
        plt.ylabel("Interaction Strength")
        plt.tight_layout()
        plt.grid(True)

    plt.suptitle(f"Interactions over Time from '{var_names[from_var_idx]}' to other variables", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

# Usage example:

# avg_interactions_time shape: [time, input_dim, input_dim], e.g. from your interaction_weights.mean(dim=0)
from_var_idx = 1  # e.g. variable 1 (0-based index)
plot_interactions_from_variable(avg_interactions_time, from_var_idx, cols_weather_multi)
